<center><h1 style="color: #555555;">Experience Script Documentation</h1></center>

<div align="center">
    <table style="width: 80%; border-collapse: collapse;">
        <tr>
            <th style="background-color: #6AAFE6; color: #ffffff; padding: 10px;">Context</th>
            <th style="background-color: #6AAFE6; color: #ffffff; padding: 10px;">Approach</th>
            <th style="background-color: #6AAFE6; color: #ffffff; padding: 10px;">Value Created</th>
        </tr>
        <tr>
            <td style="background-color: #E8F4FC; padding: 10px;">Provide the background and purpose of the script.</td>
            <td style="background-color: #E8F4FC; padding: 10px;">Describe the libraries, methods, and thought process.</td>
            <td style="background-color: #E8F4FC; padding: 10px;">Highlight the outcomes, improvements, and conclusions.</td>
        </tr>
    </table>
</div>
</html>

## STEP 1: STANDARD NORMALIZE

In [1]:
import pandas as pd

dtype = {
    'ACCOUNT_CDE': 'str',  
    'ASSETID':'str',
    'BUSINESS_UNIT':'str',
    'BUSINESS_UNIT_CDE': 'str', 
    'BUYER_ID': 'str', 
    'COMPANY_CDE': 'str',
    'COST_CENTER': 'str', 
    'COST_CENTER_DESC': 'str',
    'EXPENSE_REPORT_NAME': 'str',
    'FILE_NAME': 'str',
    'FISCAL_MONTH': 'str',
    'FISCAL_QUARTER': 'str',
    'FISCAL_YEAR': 'str',
    'INVOICE_COMMENT': 'str',
    'INVOICE_PMT_TERMS': 'str',
    'INVOICE_LINE_DESC': 'str',
    'INVOICE_LINE_NBR': 'str', 
    'INVOICE_LINE_QTY': 'str', 
    'INVOICE_LINE_UNIT_PRICE': 'str', 
    'INVOICE_NBR': 'str', 
    'INVOICE_PMT_TERMS_CDE': 'str', 
    'LEGAL_ENTITY_CDE': 'str',
    'LOCATION': 'str', 
    'LOCATION_ID': 'str',
    'PAYMENT_TYPE': 'str',
    'PO_LINE_DESC': 'str',
    'PO_LINE_ITEM_ID': 'str', 
    'PO_LINE_NBR': 'str', 
    'PO_LINE_QTY': 'str', 
    'PO_NBR': 'str', 
    'PROJECT_NAME':'str',
    'PROJECT_DESCRIPTION': 'str',
    'SUPPLIER_ADDRESS_1': 'str',
    'SUPPLIER_GRP': 'str',
    'SUPPLIER_ERP': 'str',
    'SUPPLIER_NBR': 'str',
    'SUPPLIER_NORMALIZED': 'str',
    'SUPPLIER_PMT_TERM_CDE': 'str',
    'SUPPLIER_STATE': 'str',
    'SUPPLIER_TAXID': 'str',
    'SUPPLIER_ZIP_POSTAL_CDE':'str',
    'DAYS_ATF_PO': 'str', 
    'INVOICE_LINE_AMOUNT': 'float',
    'EXCHANGE_RATE': 'float',
    'TOTAL_SPEND_USD': 'float',
    'TOTAL_SPEND':'float',
}
# Read the LQ export 
df = pd.read_csv('D:\\Users\\diana.fernandez\\Downloads\\TBC_OUTPUT_FY24_APR-MAY v.2.csv', 
encoding='UTF-8-SIG', 
encoding_errors='ignore', dtype=dtype)

print(f"DataFrame shape:{df.shape}")

df['SOURCE_DATA'].value_counts()

D:\Users\diana.fernandez\AppData\Local\Temp\1\ipykernel_8960\9107979.py:52: DtypeWarning: Columns (2,7,15,21,23,25,32,37,38,39,41,48,50,52,57,59,62,63,68,75,80,89) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('D:\\Users\\diana.fernandez\\Downloads\\TBC_OUTPUT_FY24_APR-MAY v.2.csv',


DataFrame shape:(66954, 90)


SOURCE_DATA
NONPO INVOICE    47092
Concur           10892
GNFR INVOICE      8970
Name: count, dtype: int64

In [2]:
import re

def normalize_supplier_name(row):
    word = row['SUPPLIER_ERP']

    # Check if the input is not a string
    if not isinstance(word, str):
        return word  # Return as is or modify this as needed

    # Strip '#' and anything following it from right to left
    word =re.sub(r'#.*$', '', word)

    # strip trailing numbers right to left 
    word =re.sub(r'\d+$', '', word).strip()
    
    # Remove trailing whitespaces
    word = word.rstrip()

    # For concur (credit card) data only`
    payment_methods = ['TCB*','NIC*  ','CF* ','IN *','GLF*','WPY*','ZSK*','TLF*','TN*','PY *','PLACPAY*','L2G*','IC* ', 'B2P*', 'CCI*', 
                       'CEP*', 'CKE*', 'EIG*', 'FSP*', 'HAD*', 'PAR*', 'PAYPAL *', 'PMT*', 'PP*', 'SLICE*', 'SQ *', 'TST*',
                       '4TE*', 'UEP*', 'ORACL*']
    
    # Remove listed payment methods from supplier name
    for method in payment_methods:
        if word.startswith(method):
            word = word[len(method):].strip('*').strip()
    
    word = word.replace('$', '').replace('%', '').replace('^', '').replace('#', '').replace('(', '').replace(')', '')
    word = word.replace('[', '').replace(']', '').replace(';', '').replace(':', '').replace('"', '').replace("'", "")
    word = word.replace('"', '').replace(',', '').replace('’', '').replace('-', ' ').replace('/', ' ').replace('°', '')

    to_keep = ['.COM', '.com', '.ORG', '.org', '.NET', '.net', 'WWW.', 'www.']
    if any([keep in word for keep in to_keep]):
        return word
    else:
        word = word.replace('.', '')
        return word.upper()

In [3]:
# Apply the normalization function to create the SUPPLIER_NORMALIZED column
df['SUPPLIER_NORMALIZED'] = df.apply(normalize_supplier_name, axis=1)

# Print the distinct count of supplier erp and normalized to test re results of the normalization function
print(f"Distinct values before normalization: {df['SUPPLIER_ERP'].nunique()}")
print(f"Distinct values after normalization: {df['SUPPLIER_NORMALIZED'].nunique()}")

Distinct values before normalization: 6235
Distinct values after normalization: 4236


## STEP 2: FINAL EXPORT

In [4]:
df.to_csv('D:\\Users\\diana.fernandez\\Downloads\\TBC_OUTPUT_FY24_APR-MAY v.3.csv', encoding='UTF-8-SIG', index=False)